In [15]:
import numpy as np 
import time

In [16]:
#Some controls on the matrix
#diagonal matrix
def controls(matrix):
    ret = "yes"
    matrix = np.asarray(matrix)
    if matrix.ndim != 2 or matrix.shape[0] != matrix.shape[1]:
        ret = "no"
        raise ValueError("The matrix has to be squared")

    # Only numeric
    if not np.issubdtype(matrix.dtype, np.number):
        ret = "no"
        raise TypeError("The matrix has to be numeric")

    # No NaN/Inf
    if not np.all(np.isfinite(matrix)):
        ret = "no"
        raise ValueError("The matrix does not contain NaN or Inf")

    # Check if non-diagonal elements are zero
    for i in range(matrix.shape[0]):
        for j in range(matrix.shape[1]):
            if i == j and matrix[i, j] != 0:
                ret = "no"
                break  # Exit inner loop
            elif i!=j and matrix[i, j]!=matrix[j, i]:
                ret = "no"
                break  # Exit inner loop

    return ret


In [ ]:

def compute_length_itinerary(itinerary: np.ndarray, matrix_distances: np.ndarray) -> float:
    #computes the itinerary total length
    length = 0.0
    num_cities = len(itinerary)

    # 1. Sum the distances between sequential cities
    for i in range(num_cities):
        city_now = itinerary[i]

        next_city = itinerary[(i + 1) % num_cities]

        length += matrix_distances[city_now, next_city]

    return length

def funzione_fitness(itinerary: np.ndarray, matrix_distances: np.ndarray) -> float:
    #fitness function: compute the length of the itinerary
    return compute_length_itinerary(itinerary, matrix_distances)

In [18]:
def nearest_neighbor_greedy(matrix, start=0):
    n = len(matrix)
    unvisited = list(range(n))
    path = [start]
    unvisited.remove(start) 
    while unvisited:
        last = path[-1]
        next_city = min(unvisited, key=lambda city: matrix[last, city])
        path.append(next_city)
        unvisited.remove(next_city)
    return np.array(path)


In [ ]:
def inizialize_population(dimension_population, num_cities, matrix):
    population = []
    city_ids = np.arange(num_cities)
    num_greedy = int(0.2 * dimension_population)

    # 20% Greedy
    for _ in range(num_greedy):
        # Choose a random city among ALL available cities
        random_start_node = np.random.randint(0, num_cities)
        population.append(nearest_neighbor_greedy(matrix, random_start_node))

    # 80% Random
    for _ in range(dimension_population - num_greedy):
        population.append(np.random.permutation(city_ids))
    
    return population


In [20]:
def mutation_swap(itinerary: np.ndarray, rate_mutation: float) -> np.ndarray:
    """Swap two random cities in the itinerary with a define probability"""
    mutated_itinerary= np.copy(itinerary)

    # Apply mutation with the given rate
    if np.random.rand() < rate_mutation:
        # Choose two random distinct indices
        idx1, idx2 = np.random.choice(len(itinerary), size=2, replace=False)

        # swapping the cities at the chosen indices
        mutated_itinerary[idx1], mutated_itinerary[idx2] = mutated_itinerary[idx2], mutated_itinerary[idx1]

    return mutated_itinerary

def crossover_OX(parent1, parent2):
    #Performs Ordered Crossover (OX) between two parent itineraries.
   
    n = len(parent1)
    # Choose a random segment from parent1
    start, end = sorted(np.random.choice(range(n), 2, replace=False))
    
    # Initialize child itinerary with -1
    child = [-1] * n
    # Copy the chosen segment from parent1 to the child
    child[start:end] = parent1[start:end]
    
    pos = end
    # Fill the rest of the child itinerary based on parent2's order
    for city in parent2:
        if city not in child:
            if pos >= n:
                pos = 0
            child[pos] = city
            pos += 1
    return np.array(child)

In [21]:
np.random.seed(42)
types = ["g","r1","r2"]
numbers = [10,20,50,100,200,500,1000]
for typ in types:
  for num in numbers:
        path_file = f'problem_{typ}_{num}.npy'
        try:
            array_loaded = np.load(path_file)
            to_be = controls(array_loaded)
            print(f"Is the matrix diagonal,symmetric,numeric and without Nan? {to_be}")


        except Exception as e:
             continue
        start_time = time.time()  # <--- ADDED: Start the timer
        #shape
        print(f"Shape of the array: {array_loaded.shape}")
        
        #number of cities
        NUM_CITTA = num
        matrix_distances = array_loaded
        #parameters
        DIM_POP = 200
        NUM_GENERATIONS = 800
        MUTATION_RATE = 0.2

        #population inizialization
        population = inizialize_population(DIM_POP, NUM_CITTA, matrix_distances)
        best_itinerary = None
        best_length_ever = np.inf

        for generation in range(NUM_GENERATIONS):
            #calcolate the lengths for the entire population
            lenghts = np.array([funzione_fitness(p, matrix_distances) for p in population])
            
            #Update the best value
            idx_best_current = np.argmin(lenghts)
            current_best_length = lenghts[idx_best_current]

            if current_best_length < best_length_ever:
                best_length_ever = current_best_length
                best_itinerary = population[idx_best_current]

            #conversion for selection (Roulette wheel)
            worst_length = np.max(lenghts)
            fitness_value = (worst_length - lenghts) + 1e-6 

            #normalization to get probabilities
            sum_fitness = np.sum(fitness_value)
            if sum_fitness == 0:
                probabilities = np.ones(DIM_POP) / DIM_POP
            else:
                probabilities = fitness_value / sum_fitness
            
            # start new generation
            new_population = []
           #two best
            new_population.append(population[idx_best_current])

            #new generation created
            while len(new_population) < DIM_POP:
                parent1 = population[np.random.choice(DIM_POP, p=probabilities)]
                parent2 = population[np.random.choice(DIM_POP, p=probabilities)]

                # Crossover OX
                son = crossover_OX(parent1, parent2)

                # Mutation
                mutated_son = mutation_swap(son, MUTATION_RATE)
                new_population.append(mutated_son)

            population = new_population

        # results
        minimum_length = compute_length_itinerary(best_itinerary, matrix_distances)
        end_time = time.time() # <--- ADDED: Stop the timer
        duration = end_time - start_time # <--- ADDED: Calculate the duration
       
        print(f"  BEST itinerary problem_{typ}_{num}: {best_itinerary}")
        print(f" Minimum length {minimum_length} ")
        print(f" Duration {duration} seconds ")

Is the matrix diagonal,symmetric,numeric and without Nan? yes
Shape of the array: (10, 10)
  BEST itinerary problem_g_10: [3 2 8 0 7 9 5 4 6 1]
 Minimum length 1497.6636482252907 
 Duration 6.035704612731934 seconds 
Is the matrix diagonal,symmetric,numeric and without Nan? yes
Shape of the array: (20, 20)
  BEST itinerary problem_g_20: [13  6  8  3 15 19 14 16  1 12  7  9  2 17  4 10 18  5  0 11]
 Minimum length 1756.7636359438666 
 Duration 7.212424993515015 seconds 
Is the matrix diagonal,symmetric,numeric and without Nan? yes
Shape of the array: (50, 50)
  BEST itinerary problem_g_50: [12 28 15 13 35 16 24 42 32 29 19  9 31 20 18 46 47  2  4 11  3 10 22 43
 34  7  6 44 45 23 30  5 27  0 14 25 40 48 38 49  8 17 26 39 37 21 33 36
 41  1]
 Minimum length 2905.3859892953033 
 Duration 13.746574640274048 seconds 
Is the matrix diagonal,symmetric,numeric and without Nan? yes
Shape of the array: (100, 100)
  BEST itinerary problem_g_100: [72 18 56 70 58  2 11 60 57 41 14  0 33 54 67 32 37